# Concise Implementation of Linear Regression
:label:`sec_linear_djl`

Broad and intense interest in deep learning for the past several years
has inspired both companies, academics, and hobbyists
to develop a variety of mature open source frameworks
for automating the repetitive work of implementing
gradient-based learning algorithms.
In the previous section, we relied only on
(i) `NDArray` for data storage and linear algebra;
and (ii) `GradientCollector` for calculating derivatives.
In practice, because data iterators, loss functions, optimizers,
and neural network layers (and some whole architectures)
are so common, modern libraries implement these components for us as well.

In this section, we will show you how to implement
the linear regression model from :numref:`sec_linear_scratch`
concisely by using DJL.

## Generating the Dataset

To start, we will generate the same dataset as in the previous section.

In [1]:
%use @file[../djl.json]
import ai.djl.metric.*
//import ai.djl.basicdataset.cv.classification.*
//import ai.djl.metric.*

In [2]:
class DataPoints(private val X : NDArray, private val y: NDArray) {
        
    fun getX() : NDArray? {
        return X
    }

   fun getY() : NDArray? {
        return y
    }

    // Generate y = X w + b + noise
}

fun syntheticData(manager: NDManager,w: NDArray,b: Float, numExamples: Int) : DataPoints {
       val X = manager.randomNormal(Shape(numExamples.toLong(), w.size()))
       var y = X.dot(w).add(b)
        // Add noise
        y = y.add(manager.randomNormal(0f, 0.01f, y.getShape(), DataType.FLOAT32))
        return DataPoints(X, y)
}


In [3]:
val manager = NDManager.newBaseManager()

val trueW = manager.create(floatArrayOf(2f, -3.4f))
val trueB = 4.2f

val dp = syntheticData(manager, trueW, trueB, 1000)
val features = dp.getX()
val labels = dp.getY()

## Reading the Dataset

Just like in the last section,
we can call upon DJL's `dataset` package to read data.
The first step will be to instantiate an `ArrayDataset`.
Here, we set the `features` and `labels` as parameters.
We also specify a `batchSize`
and specify a Boolean value `shuffle` indicating whether or not
we want the `ArrayDataset` to randomly sample the data.

In [4]:
// Saved in the utils file for later use
fun loadArray(features: NDArray, labels: NDArray, batchSize: Int, shuffle: Boolean) : ArrayDataset {
    return  ArrayDataset.Builder()
                  .setData(features) // set the features
                  .optLabels(labels) // set the labels
                  .setSampling(batchSize, shuffle) // set the batch size and random sampling
                  .build()
}

val batchSize = 10;
val dataset = loadArray(features, labels, batchSize, false);

To verify that it is working, we can read and print
the first minibatch of instances.

In [5]:
val batch = dataset.getData(manager).iterator().next();
val X = batch.getData().head();
val y = batch.getLabels().head();
println(X);
println(y);
batch.close();

ND: (10, 2) cpu() float32
[[ 2.2122,  1.1631],
 [ 0.774 ,  0.4838],
 [ 1.0434,  0.2996],
 [ 1.1839,  0.153 ],
 [ 1.8917, -1.1688],
 [-1.2347,  1.5581],
 [-1.771 , -0.5459],
 [-0.4514, -2.3556],
 [ 0.5794,  0.5414],
 [-1.8561,  2.6785],
]

ND: (10) cpu() float32
[ 4.6621,  4.0969,  5.2684,  6.0158, 11.9587, -3.5633,  2.5178, 11.3157,  3.505 , -8.6263]



## Defining the Model

When we implemented linear regression from scratch
(in :numref:`sec_linear_scratch`),
we defined our model parameters explicitly
and coded up the calculations to produce output
using basic linear algebra operations.
You *should* know how to do this.
But once your models get more complex,
and once you have to do this nearly every day,
you will be glad for the assistance.
The situation is similar to coding up your own blog from scratch.
Doing it once or twice is rewarding and instructive,
but you would be a lousy web developer
if every time you needed a blog you spent a month
reinventing the wheel.

For standard operations, we can use DJL's predefined blocks,
which allow us to focus especially
on the layers used to construct the model
rather than having to focus on the implementation.
To define a linear model, we first import the `Model` class,
which defines a lot of useful methods to interact with our `model`.
We will first define a model variable `model`.
We will then instantiate a SequentialBlock variable `net`, which will refer to an instance of the `SequentialBlock` class. The `SequentialBlock` class defines a container for several layers that will be chained together. Given input data, a `SequentialBlock` passes it through the first layer, in turn passing the output as the second layer’s input and so forth. In the following example, our model consists of only one layer, so we do not really need `SequentialBlock`. But since nearly all of our future models will involve multiple layers, we will use it anyway just to familiarize you with the most standard workflow.

Recall the architecture of a single-layer network as shown in :numref:`fig_singleneuron`.
The layer is said to be *fully-connected*
because each of its inputs are connected to each of its outputs
by means of a matrix-vector multiplication.
In DJL, we can use a `Linear` block to apply a linear transformation.
We simply set the number of outputs (in our case its set to 1) and choose
if we want to include a bias(yes).

![Linear regression is a single-layer neural network. ](https://resources.djl.ai/d2l-java/singleneuron.svg)
:label:`fig_singleneuron`

In [6]:
val model = Model.newInstance("lin-reg")

val net = SequentialBlock()
val linearBlock = Linear.builder().optBias(true).setUnits(1).build()
net.add(linearBlock)

model.setBlock(net)

## Defining the Loss Function

In DJL, the `Loss` class defines various loss functions.
We will use the imported class `Loss`.
In this example, we will use the DJL
implementation of squared loss (`L2Loss`).

$$
L2Loss = \sum_{i = 1}^{n}(y_i - \hat{y_i})^2
$$

L2 Loss or 'Mean Squared Error' is the sum of the squared
difference between the true `y` value and the predicted `y`
value.

In [7]:
val l2loss = Loss.l2Loss()

## Defining the Optimization Algorithm

Minibatch SGD and related variants
are standard tools for optimizing neural networks
and thus DJL supports SGD alongside a number of
variations on this algorithm through its `Optimizer` class.
When we instantiate the `Optimizer`,
we will specify the optimization algorithm we wish to use (`sgd`).
We can also manually set hyper-parameters.
SGD just requires `learningRate`,
here we set it to a fixed rate of 0.03.

In [8]:
val lrt = Tracker.fixed(0.03f)
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build()

## Instantiate Configuration and Trainer
Now we'll create a training configuration that
describes how we want to train our model.
We will then initialize a `trainer` to do the
training for us.

In [9]:
val config = DefaultTrainingConfig(l2loss)
    .optOptimizer(sgd) // Optimizer (loss function)
    .optDevices(manager.getEngine().getDevices(1)) // single GPU
    .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

val trainer = model.newTrainer(config)

## Initializing Model Parameters

Before training our model, we need to initialize the model parameters,
such as the weights and biases in the linear regression model.
We simply call the `initialize` function with the shape of the model
we are training.

In [10]:
// First axis is batch size - won't impact parameter initialization
// Second axis is the input size
trainer.initialize(Shape(batchSize.toLong(), 2))

## Metrics
Normally, DJL doesn't record metrics unless explicitly told to
as recording metrics impacts the execution flow optimizations.
To record metrics, we must instantiate `metrics` from outside
the `trainer` object and then pass it in.

In [11]:
val metrics = Metrics()
trainer.setMetrics(metrics);

## Training

You might have noticed that expressing our model through DJL
requires comparatively few lines of code.
We did not have to individually allocate parameters,
define our loss function, or implement stochastic gradient descent.
Once we start working with much more complex models,
DJL's advantages will grow considerably.
However, once we have all the basic pieces in place,
the training loop itself is strikingly similar
to what we did when implementing everything from scratch.

To refresh your memory: for some number of epochs,
we will make a complete pass over the dataset (train_data),
iteratively grabbing one minibatch of inputs
and the corresponding ground-truth labels.
For each minibatch, we go through the following ritual:

* Generate predictions, calculate loss, and calculate gradients by calling `trainBatch(batch)` (forward pass and backward pass).
* Update the model parameters by invoking the `step` function.

`Logging` will automatically print the evaluators we are watching
during each epoch.

In [12]:
val numEpochs = 3;

for (epoch in 1..numEpochs) {
    println("Epoch %d".format(epoch))
    // Iterate over dataset
    for (batch in trainer.iterateDataset(dataset)) {
        // Update loss and evaulator
        EasyTrain.trainBatch(trainer, batch);
        
        // Update parameters
        trainer.step();
        
        batch.close();
    }
    // reset training and validation evaluators at end of epoch
    trainer.notifyListeners { listener -> listener.onEpoch(trainer) }
}

Epoch 1
Training:    100% |████████████████████████████████████████| L2Loss: 3.281
Epoch 2
Training:    100% |████████████████████████████████████████| L2Loss: 4.56E-03████████████                  | L2Loss: 8.74E-03�███████            | L2Loss: 6.92E-03██████████████████████████████        | L2Loss: 6.04E-03��██    | L2Loss: 5.35E-03�████████████████████ | L2Loss: 4.80E-03
Epoch 3
Training:    100% |████████████████████████████████████████| L2Loss: 6.03E-05ining:     83% |██████████████████████████████████      | L2Loss: 6.24E-05█████████   | L2Loss: 6.05E-05██████████████████████████| L2Loss: 5.89E-05


Below, we compare the model parameters learned by training on finite data
and the actual parameters that generated our dataset.
To access parameters with DJL,
we first access the layer that we need from `model`
and then access that layer's weight and bias through its parameter list
by calling `getParameters()`.
We then simply get each param with `valueAt()`.
Here, `valueAt(0)` and `valueAt(1)` returns the weights and bias respectively.
As in our from-scratch implementation,
note that our estimated parameters are
close to their ground truth counterparts.

In [13]:
val layer = model.getBlock()
val params = layer.getParameters();
val wParam = params.valueAt(0).getArray();
val bParam = params.valueAt(1).getArray();

val w = trueW.sub(wParam.reshape(trueW.getShape())).toFloatArray();
println("Error in estimating w: [%f %f]".format(w[0], w[1]))
println("Error in estimating b: %f".format(trueB - bParam.getFloat()))

Error in estimating w: [0.000271 -0.000451]
Error in estimating b: 0.000641


## Saving Your Model
Now that you have trained your model, you probably want to save it
for future use. Additionally, you probably also want to add
metadata such as training accuracy and epochs trained.
You can do this easily. Simply point to a file location with `Paths.get`.
Metadata can be saved with the `setProperty` method.
Then call the `save` method on the model to save it!

In [14]:
val modelDir = Paths.get("../models/lin-reg")
Files.createDirectories(modelDir)

model.setProperty("Epoch", Integer.toString(numEpochs)) // save epochs trained as metadata

model.save(modelDir, "lin-reg")

model

Model (
	Name: lin-reg
	Model location: /home/ugai/d2l-java/chapter_linear-networks/../models/lin-reg
	Data Type: float32
	Epoch: 3
)

## Summary

* Using DJL, we can implement models much more succinctly.
* In DJL, the `training.dataset` package provides tools for data processing, the `nn` package defines a large number of neural network layers, and the `Loss` class defines many common loss functions.
* DJL's `training.initializer` package provides various methods for model parameter initialization.


## Exercises

1. Review the DJL documentation to see what loss functions and initialization methods are provided in the class `Loss` and `Trainer`. Replace the loss with L1 Loss.
1. How do you access the parameters during training?
